# Libraries

In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
import pyodbc
import os

## Remove default formatting in header
pd.io.formats.excel.ExcelFormatter.header_style = None

In [81]:
import warnings
warnings.filterwarnings('ignore')

# Parameters

In [2]:
year = 2022
week = 5

path = r'C:\Users\jshernandezm\OneDrive - genommalabinternacional\02Paises\Colombia\Makro\{0}\S{1}'
path = path.format(year, str(week).zfill(2))

filename = r'Stock and PO and Sales Report_Stock and PO and Sales Report.xlsx'

catalogue_products = r'C:\Users\jshernandezm\OneDrive - genommalabinternacional\02Paises\Colombia\Makro\Catalogos\maestro_productos.xlsx'

# Connections

In [3]:
conn = pyodbc.connect('Driver={SQL Server};'
                     'Server=SFEDWH01;'
                     'Trusted_Connection=yes;')

In [70]:
qdates = '''SELECT [TmpFecha]
  FROM [Gnm_DWH].[dbo].[Dim_Tiempo]
  WHERE [TmpSemanaAnioGenomma] = {1} AND TmpAnioSemanaGenomma = {0} ANd TmpDiaSemana = 7
'''

In [72]:
ddates = pd.read_sql(qdates.format(year, week), conn)

# Import Data

## SO & Stock

In [20]:
%%time
df = pd.read_excel(path + '\\' + filename)

Wall time: 1.05 s


## Catalogues

In [40]:
df_products = pd.read_excel(catalogue_products, sheet_name='Hoja2')

In [42]:
df_products = df_products[df_products['ProPstCodBarras'].notnull()]

In [43]:
df_products['ProPstCodBarras'] = df_products['ProPstCodBarras'].map(int)

df_products.drop_duplicates(inplace=True)

df_products.reset_index(drop=True, inplace=True)

C:\Users\JSHERN~1\AppData\Local\Temp/ipykernel_18880/3816992506.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_products['ProPstCodBarras'] = df_products['ProPstCodBarras'].map(int)
C:\Users\JSHERN~1\AppData\Local\Temp/ipykernel_18880/3816992506.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_products.drop_duplicates(inplace=True)


# Data Cleaning

In [18]:
# Ignore rows with no data
def get_data(data, column):
    df = data.copy()
    not_null = df[df[column].notnull()].index.min()
    df.columns = df.loc[not_null].tolist()
    df = df[not_null + 1:].copy()
    #df.dropna(axis=0, inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

In [102]:
data = get_data(df, 'Unnamed: 5')

Quitamos las últimas 4 filas ya que son nulas

In [103]:
data = data[:data.shape[0]-4]

In [104]:
data = data[['Descripción', 'Ubicación', 'Cantidad de stock', 'Cantidad de ventas']]

# Merges

## Products

In [105]:
data = data.merge(df_products, on='Descripción', how='left')

In [108]:
data[data['ProPstCodBarras'].isnull()].pivot_table(
    index='Descripción', 
    values=['Cantidad de stock', 'Cantidad de ventas'],
    aggfunc='sum'
)

,Cantidad de stock,Cantidad de ventas
Descripción,,
CAJA PILA DURACELL 9V BLISTER 48pq,0,0
CAJA PILA DURACELL C BLISTER 2uX48pq,0,0
CAJA PILA DURACELL D BLISTER 2uX48pq,0,0


data = data[data['ProPstCodBarras'].notnull()]

In [83]:
data['ProPstCodBarras'] = data['ProPstCodBarras'].map(int)

In [82]:
data['Fecha_Venta'] = ddates.loc[0].values[0]

In [85]:
data['Cadena'] = 'Makro'

In [87]:
data.rename({
    'ProPstCodBarras': 'EAN o UPC Genomma'
}, axis=1, inplace=True)

In [88]:
data['Cod_Prod de la Cadena'] = data['EAN o UPC Genomma']

In [90]:
data.rename({'Ubicación': 'Cod_Local'}, axis=1, inplace=True)

In [92]:
data.rename({'Descripción':'Descripcion_Prod'}, axis=1, inplace=True)

In [94]:
data.rename({'Cantidad de ventas':'Unidades'}, axis=1, inplace=True)

In [96]:
data[['Fecha_Venta', 'Cadena', 'Cod_Prod de la Cadena', 'EAN o UPC Genomma', 'Descripcion_Prod', 'Cod_Local', 'Unidades']]

,Fecha_Venta,Cadena,Cod_Prod de la Cadena,EAN o UPC Genomma,Descripcion_Prod,Cod_Local,Unidades
0,2022-01-30,Makro,650240035159,650240035159,ACONDICIONADOR TIO NACHO ENGROSADOR 415ml,1 - VDRIO,0
1,2022-01-30,Makro,650240035159,650240035159,ACONDICIONADOR TIO NACHO ENGROSADOR 415ml,10 - CARTG,2
2,2022-01-30,Makro,650240035159,650240035159,ACONDICIONADOR TIO NACHO ENGROSADOR 415ml,11 - CLL30,0
3,2022-01-30,Makro,650240035159,650240035159,ACONDICIONADOR TIO NACHO ENGROSADOR 415ml,12 - VVCIO,0
4,2022-01-30,Makro,650240035159,650240035159,ACONDICIONADOR TIO NACHO ENGROSADOR 415ml,13 - CALNO,0
...,...,...,...,...,...,...,...
954,2022-01-30,Makro,7798140258513,7798140258513,TRATAMIENTO TIO NACHO ULTRAHIDRATANTE COCO 200g,8 - ABOYA,0
955,2022-01-30,Makro,7798140258513,7798140258513,TRATAMIENTO TIO NACHO ULTRAHIDRATANTE COCO 200g,9 - IBAGU,2
956,2022-01-30,Makro,7798140258513,7798140258513,TRATAMIENTO TIO NACHO ULTRAHIDRATANTE COCO 200g,90000 - Cross Docking,0
957,2022-01-30,Makro,7798140258513,7798140258513,TRATAMIENTO TIO NACHO ULTRAHIDRATANTE COCO 200g,91 - TAT,0


In [98]:
data.rename({'Cantidad de stock':'Unidades Stock'}, axis=1, inplace=True)

In [99]:
data[['Fecha_Venta', 'Cadena', 'Cod_Prod de la Cadena', 'EAN o UPC Genomma', 'Descripcion_Prod', 'Cod_Local', 'Unidades Stock']]

,Fecha_Venta,Cadena,Cod_Prod de la Cadena,EAN o UPC Genomma,Descripcion_Prod,Cod_Local,Unidades Stock
0,2022-01-30,Makro,650240035159,650240035159,ACONDICIONADOR TIO NACHO ENGROSADOR 415ml,1 - VDRIO,0
1,2022-01-30,Makro,650240035159,650240035159,ACONDICIONADOR TIO NACHO ENGROSADOR 415ml,10 - CARTG,10
2,2022-01-30,Makro,650240035159,650240035159,ACONDICIONADOR TIO NACHO ENGROSADOR 415ml,11 - CLL30,0
3,2022-01-30,Makro,650240035159,650240035159,ACONDICIONADOR TIO NACHO ENGROSADOR 415ml,12 - VVCIO,0
4,2022-01-30,Makro,650240035159,650240035159,ACONDICIONADOR TIO NACHO ENGROSADOR 415ml,13 - CALNO,0
...,...,...,...,...,...,...,...
954,2022-01-30,Makro,7798140258513,7798140258513,TRATAMIENTO TIO NACHO ULTRAHIDRATANTE COCO 200g,8 - ABOYA,9
955,2022-01-30,Makro,7798140258513,7798140258513,TRATAMIENTO TIO NACHO ULTRAHIDRATANTE COCO 200g,9 - IBAGU,3
956,2022-01-30,Makro,7798140258513,7798140258513,TRATAMIENTO TIO NACHO ULTRAHIDRATANTE COCO 200g,90000 - Cross Docking,0
957,2022-01-30,Makro,7798140258513,7798140258513,TRATAMIENTO TIO NACHO ULTRAHIDRATANTE COCO 200g,91 - TAT,0


In [100]:
data['Unidades'].sum()

714

Creamos una pivot tables para exportar:

In [8]:
final = data.pivot_table(index=['Descripción'], 
                         values=['Cantidad de stock', 'Cantidad de ventas'], 
                         aggfunc='sum').reset_index()

In [9]:
final[:3]

,Descripción,Cantidad de stock,Cantidad de ventas
0,ACONDICIONADOR TIO NACHO ENGROSADOR 415ml,243,33
1,ACONDICIONADOR ANTIESPONJADO COCO KERATINA VAN...,562,28
2,ACONDICIONADOR LISO COCO KERATINA VANART 600ml,547,9


In [12]:
final.to_excel(path + '\\Makro_{0}_{1}.xlsx'.format(year, str(week).zfill(2)), 
               index=False,
               sheet_name=f'Semana {week}')

# Next Steps

- Maestra para cruzar las Descripciones y obtener los EAN
- Pedir catálogo de direcciones por PDV al cliente Makro
    - Dar de alta en el DWH las sucursales
- Hacer el cruce de productos para obtener el ProPstID
- Hacer el cruce con el catálogo de precios
- Traer la fecha